# Instagram Scraper - V2

### Importing libraries

In [1]:
#miscellanous
import os
import time
import numpy as np
import pandas as pd
import requests
from contextlib import suppress
from IPython.display import Image
from pprint import pprint

#selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
#errors
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import JavascriptException

#google cuserd vision
from google.cloud import vision
from google.cloud.vision_v1 import types

### Parameters

In [2]:
#path to project
path = r'C:\Users\danie\OneDrive\Desktop\IMT Project\\'
os.chdir(path)
baseDir = os.getcwd()

#instagram credentials
username = 'X'
password = 'X'

#google cloud vision key
key = 'X'

### Driver setup

In [3]:
def launch_driver():
    global driver
    edgeService = Service(os.path.join(baseDir, 'msedgedriver.exe'))
    driver = webdriver.Edge(service=edgeService)

def log_in(username, password):
    driver.get('https://www.instagram.com')
    
    ### accept cookies ###
    cookie_accepted = False
    while cookie_accepted != True:
        try:
            cookie_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "button[class='aOOlW  bIiDR  ']")))
            cookie_button.click()
            cookie_accepted = True
        except :
            pass
    time.sleep(1)
    
    ### fill username field ###
    username_filled = False
    while username_filled != True:
        try:
            username_field = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "input[name='username']")))
            username_field.clear()
            username_field.send_keys(username)
            username_filled = True
        except :
            pass
    time.sleep(1)
    
    ### fill password field ###
    password_filled = False
    while password_filled != True:
        try:
            password_field = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "input[name='password']")))
            password_field.clear()
            password_field.send_keys(password)
            password_filled = True
        except :
            pass
    time.sleep(1)
    
    ### clic login button ###
    login_clicked = False
    while login_clicked != True:
        try:
            login_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "button[type='submit']")))
            login_button.click()
            login_clicked = True
        except :
            pass
    time.sleep(1)
    
    ### decline remember id ###
    time.sleep(2)
    remember_declined = False
    while remember_declined != True:
        try:
            time.sleep(1)
            later_ID_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "button[class='sqdOP yWX7d    y3zKF     ']")))
            later_ID_button.click()
            remember_declined = True
        except :
            pass
    
    ### decline notifications ###
    notif_declined = False
    while notif_declined != True:
        try:
            time.sleep(1)
            later_notif_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "button[class='aOOlW   HoLwm ']")))
            later_notif_button.click()
            notif_declined = True
        except :
            pass
    time.sleep(1)

    ### accept all cookies ###
    allcookie_accepted = False
    try:    
        cookie_button_2 = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "button[class='aOOlW  bIiDR  ']")))
        cookie_button_2.click()
        allcookie_accepted = True
    except :
        pass
    time.sleep(1)

### Class : igProfile

In [4]:
class igProfile:
    def __init__(self, user_id, user_pfp=np.NaN):
        self.id_ = user_id
        self.url_ = 'https://www.instagram.com/' + user_id + '/'
        self.privacy_ = np.NaN
        self.pfp_ = user_pfp
        self.posts_ = np.NaN
        self.nposts_ = np.NaN
        self.faces_ = np.NaN
    
    def __repr__(self):
        return f'igProfile({self.id_})'
    
#   def __str__(self):
#       return f'''id: {self.id_} \nprivacy: {self.privacy_} \nurl: {self.url_} \npfp: {self.pfp_} \nfaces: {self.faces_} \nnposts: {self.nposts_} \nlast post: {self.posts_[-1].id_}'''
    
    def get_posts(self):
        if driver.current_url != self.url_:
            driver.get(self.url_)
        posts = list()
        for post_ref in driver.find_elements(By.CSS_SELECTOR, ".v1Nh3.kIKUG._bz0w>a"):
            post_url = post_ref.get_attribute('href')
            post_id = post_url.split('/p/')[1][:-1]
            posts.append(post_id)
        self.posts_ = [igPost(post_id=post, user_id=self.id_) for post in posts]
        self.nposts_ = int(''.join(driver.find_element(By.CSS_SELECTOR, 'span[class="g47SY "]').text.split(',')))
        return posts
    
    def get_pfp(self):
        if driver.current_url != self.url_:
            driver.get(self.url_)
        try: #private account
            pfp = driver.find_element(By.CSS_SELECTOR, 'img[class="be6sR"]').get_attribute('src')
            self.privacy_ = 'private'
        except NoSuchElementException: #public account
            pfp = driver.find_element(By.CSS_SELECTOR, 'img[class="_6q-tv"]').get_attribute('src')
            self.privacy_ = 'public'
        self.pfp_ = pfp
        return pfp
    
    def analyze_pfp(self):
        self.faces_ = analyze_pic(self.pfp_)
        return self.faces_

### Class : igPost

In [5]:
class igPost:
    def __init__(self, post_id, user_id=np.NaN):
        self.id_ = post_id
        self.url_ = 'https://www.instagram.com/p/' + post_id + '/'
        self.user_id_ = user_id
        self.user_ = igProfile(user_id)
        self.nlikes_ = np.NaN
        self.likers_ = np.NaN
        self.status_ = np.NaN
        self.date_ = np.NaN
        
    def __repr__(self):
        return f'igPost({self.id_})'
    
#   def __str__(self):
#       return f'''id: {self.id_} \nurl: {self.url_} \ndate: {self.date_} \nuser: {self.user_id_} \nnlikes: {self.nlikes_}'''
    
    def get_user(self):
        if driver.current_url != self.url_:
            driver.get(self.url_)
        try:    
            user_id = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "a[class='sqdOP yWX7d     _8A5w5   ZIAjV ']"))).text
            self.user_id_ = user_id
            self.user_ = igProfile(user_id)
            return igProfile(user_id)
        except :
            print(f"cannot fetch {self.id_}'s user")
            return np.NaN
        
    def get_likes(self):
        print(50*'*')
        likers = list()

        if driver.current_url != self.url_ or self.user_ == np.NaN:
            self.user_ = self.get_user()
        
        time.sleep(1)
        print('> extracting post date')
        date = driver.find_element(By.CSS_SELECTOR, 'time[class="_1o9PC Nzb55"]').get_attribute('datetime').split('T')[0]
        date = date.split('-')[::-1]
        self.date_ = '-'.join(date)
        print(f'> date extracted ({self.date_})')
        
        t_ini = time.time()
        
        ### get number of likes ###
        try: #list of likes available
            nlikes = driver.find_element(By.CSS_SELECTOR, 'a[class="zV_Nj"]').text.split()[:-1][0].replace(',', '')
            self.nlikes_ = int(nlikes)
            print(f'> post {self.id_} from {self.user_id_} has {self.nlikes_} like(s)')
        except: #list of likes unavailable
            views_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "span[class='vcOH2']")))
            views_button.click()
            time.sleep(1)
            nlikes = driver.find_element(By.CSS_SELECTOR, 'div[class="vJRqr"]').text.split()[:-1][0].replace(',', '')
            self.nlikes_ = int(nlikes)
            print(f'> post {self.id_} from {self.user_id_} has {self.nlikes_} like(s)')
            print(f'> likes are unavailable')
            self.status_ = 'unavailable'
            return np.NaN
        
        clicked=False
        while clicked==False:
            try:
                print('> locating like button')
                likes_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable(
                    (By.CLASS_NAME, "zV_Nj")))
                time.sleep(3)
                print('> like button located')
                likes_button.click()
                clicked = True
                print('> like button clicked')
            except ElementClickInterceptedException:
                time.sleep(1)
                print('> failure to locate button')
        
        print('> locating like popup window')
        popup = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, 'div[style="height: 356px; overflow: hidden auto;"]')))
        print('> like popup located')
        time.sleep(3)
        
        t = 0
        while not likers:
            time.sleep(t)
            t = t + 2
        
###
            block = False
            start = False

            while True: #ou block == True ???
                while True:
                    if start == False:
                        break
                    try:
                        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + 1000;', popup)
                        print('> scroll down')
                        break
                    except StaleElementReferenceException:
                        time.sleep(1)

                start = True

                time.sleep(0.05)
                likers_temp = driver.find_elements(By.CSS_SELECTOR, 'a[class="FPmhX notranslate MBL3Z"]')
                new_liker = False

                for liker_id in likers_temp:
                    liker_id = liker_id.get_attribute('title')
                    string = f"{liker_id}'s profile picture"
                    pfp = driver.find_element(By.CSS_SELECTOR, 'img[alt="%s\"]' % string).get_attribute('src')
                    liker = igProfile(user_id=liker_id, user_pfp=pfp)
                    m = len(likers)
                    m = max(m-50,0)
                    if any(x.id_ == liker.id_ for x in likers[m:]) == False: # if unseen liker
                        likers.append(liker)
                        new_liker = True
                        block = False
                        print(f'{len(likers)})\t user id: {liker.id_}')

                if block == True:
                    break

                with suppress(JavascriptException):
                    if new_liker == False:
                        block = True
                        print('> scrolling blocked')
                        print('> attempt to unblock scrolling')
                        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop - 500;', popup)
                        time.sleep(0.1)
                        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + 1000;', popup)
                        time.sleep(0.1)
                        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop - 500;', popup)
                        time.sleep(0.1)
                        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + 1000;', popup)
                        time.sleep(0.1)
                        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop - 500;', popup)
                        time.sleep(0.1)
                        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + 2000;', popup)
                        time.sleep(0.1)
                        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + 100;', popup)
                        time.sleep(0.1)
            print('> popup bottom reached')
        self.likers_ = likers
        self.status_ = 'available'
        print(f'> {len(self.likers_)}/{self.nlikes_} likers were scraped in {np.round(time.time()-t_ini,2)} sec')
        return likers
###

### Journals

In [34]:
class igJournal(igProfile):
    def __init__(self, user_id):
        super().__init__(user_id)

journals_dict = {'lequipe': "L'Équipe",
                 'tf1': 'TF1',
                 'lemondefr': 'Le Monde',
                 'europe1': 'Europe 1',
                 'm6officiel': 'M6',
                 'liberationfr': 'Libération',
                 'rtl_france': 'RTL',
                 'lefigarofr': 'Le Figaro',
                 '20minutesfrance': '20 Minutes',
                 'bfmtv': 'BFMTV',
                 'leparisien': 'Le Parisien',
                 'lobs': "L'Obs",
                 'cnewsofficiel': 'CNEWS',
                 'lhumanitefr': "L'Humanité",
                 'lexpressfr': "L'Express",
                 'ellefr': 'Elle',
                 'closer_france': 'Closer',
                 'buzzfeed': 'Buzfeed',
                 'brutofficiel': 'Brut',
                 'vice_france': 'Vice',
                 'ajplusfrancais': 'AJ+',
                 'slatefrance': 'Slate',
                 'mediapart': 'Mediapart',
                 'mariannelemag': 'Marianne',
                 'valeurs_actuelles': 'Valeurs Actuelles',
                 'charlie_hebdo_officiel': 'Charlie Hebdo',
                 'scienceetvie': 'Science et Vie',
                 'latribune_news': 'La Tribune',
                 'letudiant_': "L'Étudiant",
                 'francesoirfr': 'FranceSoir',
                 'lepointfr': 'Le Point',
                 'courrierinter': 'Courrier International'}

journals = [igJournal(j_id) for j_id in journals_dict.keys()]

In [35]:
def init_gcv(key):
    global image, client
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key
    client =  vision.ImageAnnotatorClient()
    image = types.Image()
    
def analyze_pic(url):
    image.source.image_uri = url
    response_face = client.face_detection(image=image)
    nb_face = len(response_face.face_annotations)
    if nb_face >= 1:
        confidence = response_face.face_annotations[0].detection_confidence
        print(f'{str(nb_face)} face(s) detected ({np.round(confidence, 3)} confidence rate)')
        return nb_face
    elif nb_face == 0:
        print('No face detected')
        return 0

def analyze_test(url):
    return 'test'

In [36]:
init_gcv(key)

In [11]:
launch_driver()
log_in(username, password)

In [12]:
user = igProfile(user_id = 'danny.wo')
user.get_posts()
user.get_pfp()

print('> profile id \t\t', user.id_)
print('> privacy \t\t', user.privacy_)
print('> profile url \t\t', user.url_)
#print('> latest posts \t\t', ', '.join(user.posts_[:5]))
#print('> profile picture url \t', user.pfp_)
Image(requests.get(user.pfp_).content, width=250, height=250)
post = user.posts_[0]
screening = post.get_likes()

> profile id 		 danny.wo
> privacy 		 public
> profile url 		 https://www.instagram.com/danny.wo/
**************************************************
> extracting post date
> date extracted (22-11-2021)
> post CWl86YDo9DI from danny.wo has 27 like(s)
> locating like button
> like button located
> like button clicked
> locating like popup window
> like popup located
1)	 user id: alexia.plrd
2)	 user id: samy2p1
3)	 user id: _insn_
4)	 user id: inesbouam
5)	 user id: cemosito
6)	 user id: adlafach
7)	 user id: poucekolosso
8)	 user id: momolebgdefou
9)	 user id: arthurfourcadet
10)	 user id: loic.bigeard
11)	 user id: nicolas_girard
> scroll down
12)	 user id: _rayanegrch
13)	 user id: yanis_kht
14)	 user id: thuvar93
15)	 user id: hugokerrest
16)	 user id: omar_cherifz
17)	 user id: najibnts
18)	 user id: erwan55000
> scroll down
19)	 user id: dm0suu
20)	 user id: backboii_
21)	 user id: swann.al1
22)	 user id: zaki.brm
23)	 user id: victographe
24)	 user id: edenlb_
25)	 user id: lou_vr

In [13]:
db_cols = ['timestamp',
           'post_id', 
           'journal', 
           'post_date', 
           'post_url', 
           'post_nlikes', 
           'n_scraped', 
           'liker_id', 
           'liker_privacy', 
           'liker_url', 
           'liker_pfp', 
           'pfp_faces'
          ]

df = pd.DataFrame(columns=db_cols)

In [15]:
journal = igJournal('rtl_france')

journal.get_posts()
for post in journal.posts_[:2]:
    timestamp = pd.Timestamp.now()
    screening = post.get_likes()
    if type(screening) != float:
        for liker in screening:
            data = {'timestamp': timestamp,
                    'post_id': post.id_, 
                    'journal': post.user_id_, 
                    'post_date': post.date_, 
#                    'post_url': post.url_,
                    'post_nlikes': post.nlikes_, 
                    'n_scraped':len(post.likers_), 
                    'liker_id': liker.id_, 
                    'liker_privacy': liker.privacy_, 
#                    'liker_url': liker.url_, 
                    'liker_pfp': liker.pfp_, 
                    'pfp_faces': liker.faces_
                   }
            df = df.append(data, ignore_index=True)
    else:
        pass

display(df)

In [24]:
df[df['liker_id']=='denis_b18230']['liker_pfp'].values

array(['https://scontent-cdg2-1.cdninstagram.com/v/t51.2885-19/s150x150/195584084_308845047404292_2136187259709961063_n.jpg?_nc_ht=scontent-cdg2-1.cdninstagram.com&_nc_cat=100&_nc_ohc=koZIahxX8DEAX-aa1FU&tn=kKDWb8bAB8YOb6m7&edm=AAo1ks0BAAAA&ccb=7-4&oh=c7332405c10c2ae8c50c020744411d3d&oe=61B1C11B&_nc_sid=01e9e1'],
      dtype=object)

In [ ]:
df['post_id'].value_counts()

In [ ]:
df['pfp_faces'] = df['liker_pfp'].apply(analyze_test)

In [47]:
df

,timestamp,post_id,journal,post_date,post_url,post_nlikes,n_scraped,liker_id,liker_privacy,liker_url,liker_pfp,pfp_faces
0,2021-12-03 10:07:40.851164,CXAtaNqtWkc,rtl_france,03-12-2021,NaN,77,77,denis_b18230,NaN,NaN,https://scontent-cdg2-1.cdninstagram.com/v/t51...,NaN
1,2021-12-03 10:07:40.851164,CXAtaNqtWkc,rtl_france,03-12-2021,NaN,77,77,59500jpv,NaN,NaN,https://scontent-cdt1-1.cdninstagram.com/v/t51...,NaN
2,2021-12-03 10:07:40.851164,CXAtaNqtWkc,rtl_france,03-12-2021,NaN,77,77,eforveille,NaN,NaN,https://scontent-cdg2-1.cdninstagram.com/v/t51...,NaN
3,2021-12-03 10:07:40.851164,CXAtaNqtWkc,rtl_france,03-12-2021,NaN,77,77,ivan.debs,NaN,NaN,https://scontent-cdt1-1.cdninstagram.com/v/t51...,NaN
4,2021-12-03 10:07:40.851164,CXAtaNqtWkc,rtl_france,03-12-2021,NaN,77,77,moussejeanpierre,NaN,NaN,https://scontent-cdg2-1.cdninstagram.com/v/t51...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
72,2021-12-03 10:07:40.851164,CXAtaNqtWkc,rtl_france,03-12-2021,NaN,77,77,davidguerinofficiel,NaN,NaN,https://scontent-cdt1-1.cdninstagram.com/v/t51...,NaN
73,2021-12-03 10:07:40.851164,CXAtaNqtWkc,rtl_france,03-12-2021,NaN,77,77,msaliceurban,NaN,NaN,https://scontent-cdt1-1.cdninstagram.com/v/t51...,NaN
74,2021-12-03 10:07:40.851164,CXAtaNqtWkc,rtl_france,03-12-2021,NaN,77,77,fantastique_kati,NaN,NaN,https://scontent-cdt1-1.cdninstagram.com/v/t51...,NaN
75,2021-12-03 10:07:40.851164,CXAtaNqtWkc,rtl_france,03-12-2021,NaN,77,77,lesgrossestetes,NaN,NaN,https://scontent-cdg2-1.cdninstagram.com/v/t51...,NaN
